# Estimate diffusivity parameters from 1D b-space optimization
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import time
from IPython import display
import obsfit1d
from scipy.interpolate import interp1d

In [ ]:
# 2D run data:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
run = [basedir + 'z0_0p2500_AH_000_Kinf_m5_ADV_2_slope_400/'];

params = np.load(run[0] + 'runparams.npz');locals().update(params)
b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
ba = (b[:-1]+b[1:])/2

with h5py.File(run[0] + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    B = (B[1:,:] + B[:-1,:])/2
    B = (B[:,1:] + B[:,:-1])/2
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

with h5py.File(run[0] + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    trF = np.zeros((len(b)-1,len(t)))
    for i in range(0,len(t)):#len(t),pfr):
        tr = file['tasks']['tr'][i,:,:]
        tr = (tr[1:,:] + tr[:-1,:])/2
        tr = (tr[:,1:] + tr[:,:-1])/2
        tr = tr*dA
        trF[:,i] = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])

zF = ba/N2;
dzF = zF[1]-zF[0]
tF = t;

imax = np.argmax(trF[:,0])
zF = zF - zF[imax]
# trF = trF/np.sum(trF[imax,0]*dzF)


In [ ]:
tind = (len(trF[0,:])-1)
csum = np.cumsum(trF[:,tind])
f = interp1d(csum,zF)
xmax = f(0.99*csum[-1])+500.
xmin = f(0.01*csum[-1])-500.
plt.plot(trF[:,tind],zF,'-k')
plt.plot(0.,xmax,'xr')
plt.plot(0.,xmin,'xr')


In [ ]:
tind = (len(trF[0,:])-1)
res = obsfit1d.fit3par(zF,trF[:,tind],int(tF[tind]*1e5),sz)
#resMN = obsfit1d.fit3par(zF,trF[:,tind],int(tF[tind]*1e5),sz)
obsfit1d.plot(res,zF,trF[:,tind],int(tF[tind]*1e5),sz)

In [ ]:
import imp
imp.reload(obsfit1d)